### INSTRUCTIONS - Assignment 1

- Submit only the ipynb file with file format main_{UBID}.ipynb (e.g. main_50370999.ipynb), Submission other than this format will get zero.
- Don't submit the zip folder.
- We will run all the cells from start to finish, if we get any error or cell stuck for more than 20 sec. will get zero for that problem. So make sure ipynb file run from start to finish before submitting.

- Refrain from using Sklearn library unless explicitly mentioned.

- Don't import additional libraries in the file.


In [ ]:
# imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

#### STEP 1 - Load Data

In [ ]:
# loads csv file into pandas dataframe


df = pd.read_csv('IRIS_data.csv',  dtype={'sepal_length': np.float64,'sepal_width': np.float64,'petal_length': np.float64,'petal_width': np.float64})







#### STEP 2 - 
Split the Data into different training subsets and testing split 

( 40% Training - Subset1, 40% Training - Subset2 and 20% Testing) 

Hint: The train_test split with stratify function is already imported earlier in the notebook

( 5 Points)

In [ ]:




x = df
y = df["species"]

#  Now here we will split the data to training 80% and testing 20 %
x_full_train,x_full_test,y_full_train,y_full_test = train_test_split(x,y, test_size=0.2, random_state=None, shuffle=True,stratify=y)




#  Now as per requirment we will split the training set again so we have the required subset of training which will make our final breakdown as:  40% Training-Subset 1 ,40% Training-Subset 2, 20% testing

x_subset_1,x_subset_2,y_subset_1,y_subset_2 = train_test_split(x_full_train,y_full_train, test_size=0.5, random_state=None, shuffle=True,stratify=y_full_train) 





### Perform Steps 3 to 7 for each subset

#### STEP 3 - Calculate the posterior probability for each class (20 Points)

Calculate the prior probability and likelihood, followed by evidence calculation and finally the posterior probability using Bayes Rule

You can write everything in the cell below or add more cells as needed especially if you want to write a helper function which you use to do this

In [ ]:
# Helper Function  that will calculate likelihood , prior , action , and will also getmeanVec
def likelihood(xvec, meanvec, covmat, dims = 4):
    d = xvec - meanvec
    a = 1 / float(((2 * np.pi) ** (float(dims)/2)) * (np.linalg.det(covmat)**(0.5)))
    expon = -(0.5) * np.matmul(np.transpose(d), np.matmul(np.linalg.inv(covmat), d))
    expon =  np.exp(expon)

    likelihood = a * expon

    return likelihood

def getPrior(reqClass, class1, class2,class3):
    #for binary classifiers
    return float(len(reqClass[0]))/float(len(class1[0]) + len(class2[0]) + len(class3[0]))

def action(p1, p2,p3):
    #for binary classifiers
    list = [p1,p2,p3]
    maximum = max(list)
    return maximum   
      

def getmeanVec(train):
    n = len(train)
    meanvec = np.zeros(n)
    for i in range(len(train)):
        meanvec[i] = np.mean(train[i])
    return meanvec




In [ ]:
# Helper Function that will calculate probability
# Here my action will return the maximum posterior probability and on based of maximum posterior probability It ll classify that into state. Class is being referenced as state
def getProbabilities(row, t1, t2,t3):

    train1 = np.asarray(t1.values.tolist()).transpose()
    train2 = np.asarray(t2.values.tolist()).transpose()
    train3 = np.asarray(t3.values.tolist()).transpose()

    cov1 = np.cov(train1)
    cov2 = np.cov(train2)
    cov3 = np.cov(train3)
    

    meanvec1 = getmeanVec(train1)
    meanvec2 = getmeanVec(train2)
    meanvec3 = getmeanVec(train3)

    prior1 = getPrior(train1,train1, train2, train3)
    prior2 = getPrior(train2,train1, train2, train3)
    prior3 = getPrior(train3,train1, train2, train3)

    p1 = likelihood(row, meanvec1, cov1,4)
    p2 = likelihood(row, meanvec2, cov2,4)
    p3 = likelihood(row, meanvec3, cov3,4)

    evidence = p1 * prior1 + p2 * prior2 + p3 * prior3

    prob1 = p1 * prior1 /evidence
    prob2 = p2 * prior2/evidence
    prob3 = p3 * prior3/evidence


    # Category will return max posterior probability

    category = action(prob1,prob2,prob3)
    state = 0

    
    if category == prob1:
        state =  1
    elif category == prob2:
        state =  2
    elif category == prob3:
        state =  3
  

    

    data = [row,prior1,prior2,prior3,p1,p2,p3,evidence,prob1,prob2,prob3,state]

    return data

    

In [ ]:
#STEP 3 
# This is the main function that will print posterior proobability and will put that posterior probability in dataframe for every row in test set. The data frame will have all the posterior probabilities with all details.
# This will give us the posteror Frame for training set 1 which is x_subset_1
# I ll be considering class 1 as Iris-VersiColor class 2 as Iris-virginica and class 3 as IRis Setosa
# I am adding one more column class which will be decided based on maximum posterior probability


train1 = x_subset_1.loc[x_subset_1['species'] == 'Iris-versicolor']
train2 = x_subset_1.loc[x_subset_1['species'] == 'Iris-virginica']
train3 = x_subset_1.loc[x_subset_1['species'] == 'Iris-setosa']

train1 = train1.drop('species',axis =1)
train2 = train2.drop('species',axis =1)
train3 = train3.drop('species',axis =1)


test = x_full_test.drop('species',axis =1)



       

temp= list()
for row in test.values:
       data = getProbabilities(row, train1, train2,train3)
       temp.append(data)

#myframe = pd.DataFrame( temp, columns=['row','prior1','prior2','prior3','p1','p2','p3','evidence','prob1','prob2','prob3'])


myPosteriorFrame1 = pd.DataFrame( temp, columns=['row','prior of 1','prior of 2','prior of 3','likelihood of 1','likelihood of 2','likelihood of 3','evidence','PosteriorProbability1','PosteriorProbability2','PosteriorProbability3','Class'])

display(myPosteriorFrame1)




In [ ]:
#STEP 3 
# This is the main function that will calculate posterior proobability and will put that posterior probability in dataframe for every row. The data frame will have all the posterior probabilities with all details.
# This will give us the posteror Frame for training set 2 which is x_subset_2



train1 = x_subset_2.loc[x_subset_2['species'] == 'Iris-versicolor']
train2 = x_subset_2.loc[x_subset_2['species'] == 'Iris-virginica']
train3 = x_subset_2.loc[x_subset_2['species'] == 'Iris-setosa']

train1 = train1.drop('species',axis =1)
train2 = train2.drop('species',axis =1)
train3 = train3.drop('species',axis =1)


test = x_full_test.drop('species',axis =1)



       

temp= list()
for row in test.values:
       data = getProbabilities(row, train1, train2,train3)
       temp.append(data)

#myframe = pd.DataFrame( temp, columns=['row','prior1','prior2','prior3','p1','p2','p3','evidence','prob1','prob2','prob3'])


myPosteriorFrame2 =  pd.DataFrame( temp, columns=['row','prior of 1','prior of 2','prior of 3','likelihood of 1','likelihood of 2','likelihood of 3','evidence','PosteriorProbability1','PosteriorProbability2','PosteriorProbability3','Class'])

display(myPosteriorFrame2)






#### STEP 4 - Calculate probability of error for each class (10 Points)

Please refer the slides for this section.

In [ ]:
#STEP 4
# In this step I will display the probability of error for output test set for training set 1 which is myPosteriorFrame1
# I am taking probability of error for each class as (1-Probability of correctness/(Max. Posterior Probability)
# Here I will display the already calculated error for each class


errorFrame = myPosteriorFrame1.copy()

rows = errorFrame.shape[0]

errorList = list()

for row in range(rows):
    error = 1-(max(errorFrame.iloc[row]["PosteriorProbability1"],errorFrame.iloc[row]["PosteriorProbability2"],errorFrame.iloc[row]["PosteriorProbability3"]))
    errorList.append(error)


errorFrame["Error"] = errorList


display(errorFrame)








In [ ]:
#STEP 4
# In this step I will display the probability of error for output of test set for training set 2 which is myPosteriorFrame2
# I am taking probability of error for each class as (1-Probability of correctness(Max. Posterior Probability)
# Here I will display the  calculated error for each class. Error will be calculated by 1 - correctness

errorFrame = myPosteriorFrame2.copy()

rows = errorFrame.shape[0]

errorList = list()

for row in range(rows):
    error = 1-(max(errorFrame.iloc[row]["PosteriorProbability1"],errorFrame.iloc[row]["PosteriorProbability2"],errorFrame.iloc[row]["PosteriorProbability3"]))
    errorList.append(error)



errorFrame["Error"] = errorList

display(errorFrame)


#### STEP 5 - Caculate the conditional risk (30 Points)
 5.1 Consider the range for calculation of lambda between {0, 0.25, 0.5, 0.75, 1}
 
 5.2 Caclulate the Conditional Risk for every class (10 Points)
 
 5.3 Caclulate the overall risk (10 Points)
 
 5.4 Calculate the lambda for minimum Bayes' Risk.(10 Points)
 
 Note: you may want to use the code discussed in class, but you need to be able to extend the functions like action() so that loss components can be used to weigh the corresponding postrior probability to compute the risks.

In [ ]:
# STEP 5
# Step 5 will be performed two times for test set: once for posterior probabilities of test set when we took  training as training set1 which is x_subset_1
# since we already have those pposterior probabilities in our frame which is myPosteriorFrame1
# ConditionalRisk1 is for class chosen 1 Conditional Risk 2 is for class 2 and Simmillarly for 3
# After Printing the frame with all conditional risks and mimimum risk I will print overall risk for every lamda below it at last in the cell
# Lamda for minimum RIsk will be displayed after overall risk for all ranges for lamda has been calculated. 




# lambda is 1 for myPosteriorFrame1

lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 1
lossClass1GivenClass3 = 1
lossClass2GivenClass1 = 1
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 1
lossClass3GivenClass1 = 1
lossClass3GivenClass2 = 1
lossClass3GivenClass3 = 0

myframe = myPosteriorFrame1.copy()

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)
    #print(currentConditionalRisk)

myframe["ConditionalRisk1Lamda1"] = conditionalRisk1
myframe["ConditionalRisk2Lamda1"] = conditionalRisk2
myframe["ConditionalRisk3Lamda1"] = conditionalRisk3

minRiskList = list()
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda1"],myframe.iloc[i]["ConditionalRisk2Lamda1"],myframe.iloc[i]["ConditionalRisk3Lamda1"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)


myframe["MinRiskLamda1"] = minRiskList



myRiskFrame1 = myframe[["row","ConditionalRisk1Lamda1","ConditionalRisk2Lamda1","ConditionalRisk3Lamda1","MinRiskLamda1"]]
display(myRiskFrame1)



#Now I will be printing the overall risk 
overallRiskLamda1 = myframe["MinRiskLamda1"].sum()

print("The overall risk for Lamda is 1 is below:")
display(overallRiskLamda1)













In [ ]:
# STEP 5 lambda is 0.75 for myPosteriorFrame1
lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 0.75
lossClass1GivenClass3 = 0.75
lossClass2GivenClass1 = 0.75
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 0.75
lossClass3GivenClass1 = 0.75
lossClass3GivenClass2 = 0.75
lossClass3GivenClass3 = 0

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)

myframe["ConditionalRisk1Lamda0.75"] = conditionalRisk1
myframe["ConditionalRisk2Lamda0.75"] = conditionalRisk2
myframe["ConditionalRisk3Lamda0.75"] = conditionalRisk3

minRiskList = list()
'''
    for i in range(rows):
        riskList = [myframe.iloc[i]["conditionalRisk1"],myframe.iloc[i]["conditionalRisk2"],myframe.iloc[i]["conditionalRisk3"]]
        minRisk = min(riskList)
        if minRisk == myframe.iloc[i]["conditionalRisk1"]:
            minRiskList.append(1)
        elif  minRisk == myframe.iloc[i]["conditionalRisk2"]:
            minRiskList.append(2)
        elif  minRisk == myframe.iloc[i]["conditionalRisk3"]:
            minRiskList.append(3)   
'''
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda0.75"],myframe.iloc[i]["ConditionalRisk2Lamda0.75"],myframe.iloc[i]["ConditionalRisk3Lamda0.75"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)





myframe["MinRiskLamda0.75"] = minRiskList


myRiskFrame2 = myframe[["row","ConditionalRisk1Lamda0.75","ConditionalRisk2Lamda0.75","ConditionalRisk3Lamda0.75","MinRiskLamda0.75"]]
display(myRiskFrame2)





overallRiskLamda75= myframe["MinRiskLamda0.75"].sum()

print("The overall risk for Lamda is 0.75 is below:")
display(overallRiskLamda75)







In [ ]:
# STEP 5 lambda is 0.5 for myPosteriorFrame1
lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 0.5
lossClass1GivenClass3 = 0.5
lossClass2GivenClass1 = 0.5
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 0.5
lossClass3GivenClass1 = 0.5
lossClass3GivenClass2 = 0.5
lossClass3GivenClass3 = 0

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)
    #print(currentConditionalRisk)

myframe["ConditionalRisk1Lamda0.5"] = conditionalRisk1
myframe["ConditionalRisk2Lamda0.5"] = conditionalRisk2
myframe["ConditionalRisk3Lamda0.5"] = conditionalRisk3

minRiskList = list()
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda0.5"],myframe.iloc[i]["ConditionalRisk2Lamda0.5"],myframe.iloc[i]["ConditionalRisk3Lamda0.5"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)       



myframe["MinRiskLamda0.5"] = minRiskList


myRiskFrame3 = myframe[["row","ConditionalRisk1Lamda0.5","ConditionalRisk2Lamda0.5","ConditionalRisk3Lamda0.5","MinRiskLamda0.5"]]
display(myRiskFrame3)



overallRiskLamda5= myframe["MinRiskLamda0.5"].sum()

print("The overall risk for Lamda is 0.5 is below:")
display(overallRiskLamda5)


In [ ]:
# STEP 5 lambda is 0.25 for myPosteriorFrame1
lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 0.25
lossClass1GivenClass3 = 0.25
lossClass2GivenClass1 = 0.25
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 0.25
lossClass3GivenClass1 = 0.25
lossClass3GivenClass2 = 0.25
lossClass3GivenClass3 = 0

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)
    #print(currentConditionalRisk)

myframe["ConditionalRisk1Lamda0.25"] = conditionalRisk1
myframe["ConditionalRisk2Lamda0.25"] = conditionalRisk2
myframe["ConditionalRisk3Lamda0.25"] = conditionalRisk3

minRiskList = list()
minRiskList = list()
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda0.25"],myframe.iloc[i]["ConditionalRisk2Lamda0.25"],myframe.iloc[i]["ConditionalRisk3Lamda0.25"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)         



myframe["MinRiskLamda0.25"] = minRiskList


myRiskFrame4= myframe[["row","ConditionalRisk1Lamda0.25","ConditionalRisk2Lamda0.25","ConditionalRisk3Lamda0.25","MinRiskLamda0.25"]]
display(myRiskFrame4)

overallRiskLamda25= myframe["MinRiskLamda0.25"].sum()

print("The overall risk for Lamda is 0.25 is below:")
display(overallRiskLamda25)


In [ ]:
# Printind the lamda with minimum Bayes Risk for myPosteriorFrame1
# Here I will be taking overall risk for all lamdas and the lamda whose output will be minimum among all lamdas will be my answer
# Here I am taking minimum risk corrosponding to every lamda , and comparing those minimum values and choosing the lamda with lowest risk for every row 


minimum= overallRiskLamda1
minLamda = 1



if(overallRiskLamda75 < minimum):
    minimum = overallRiskLamda75
    minLamda = 0.75
if(overallRiskLamda5 < minimum):
    minimum = overallRiskLamda5
    minLamda = 0.5
if(overallRiskLamda25 < minimum):
    minLamda = 0.25
    minimum = overallRiskLamda25    

    
print("The Lamda for minimum Bayes Risk is",minLamda)

In [ ]:
# STEP 5
# Step 5 will be performed two times for test set: once for posterior probabilities of test set when we took  training as training set1 which is x_subset_2
# since we already have those pposterior probabilities in our frame which is myPosteriorFrame2

# lambda is 1 for myPosteriorFrame2

lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 1
lossClass1GivenClass3 = 1
lossClass2GivenClass1 = 1
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 1
lossClass3GivenClass1 = 1
lossClass3GivenClass2 = 1
lossClass3GivenClass3 = 0

myframe = myPosteriorFrame2.copy()

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)
    #print(currentConditionalRisk)

myframe["ConditionalRisk1Lamda1"] = conditionalRisk1
myframe["ConditionalRisk2Lamda1"] = conditionalRisk2
myframe["ConditionalRisk3Lamda1"] = conditionalRisk3

minRiskList = list()
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda1"],myframe.iloc[i]["ConditionalRisk2Lamda1"],myframe.iloc[i]["ConditionalRisk3Lamda1"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)


myframe["MinRiskLamda1"] = minRiskList



myRiskFrame1 = myframe[["row","ConditionalRisk1Lamda1","ConditionalRisk2Lamda1","ConditionalRisk3Lamda1","MinRiskLamda1"]]

display(myRiskFrame1)

overallRiskLamda1= myframe["MinRiskLamda1"].sum()

print("The overall risk for Lamda is 1is below:")
display(overallRiskLamda1)

#display(myframe)
#print("Overall risk is ",overallRisk)


overallRiskLamda1= myframe["MinRiskLamda1"].sum()

print("The overall risk for Lamda is 1 is below:",overallRiskLamda1)








In [ ]:
# STEP 5 lambda is 0.75 for myPosteriorFrame2
lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 0.75
lossClass1GivenClass3 = 0.75
lossClass2GivenClass1 = 0.75
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 0.75
lossClass3GivenClass1 = 0.75
lossClass3GivenClass2 = 0.75
lossClass3GivenClass3 = 0

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)

myframe["ConditionalRisk1Lamda0.75"] = conditionalRisk1
myframe["ConditionalRisk2Lamda0.75"] = conditionalRisk2
myframe["ConditionalRisk3Lamda0.75"] = conditionalRisk3

minRiskList = list()
'''
    for i in range(rows):
        riskList = [myframe.iloc[i]["conditionalRisk1"],myframe.iloc[i]["conditionalRisk2"],myframe.iloc[i]["conditionalRisk3"]]
        minRisk = min(riskList)
        if minRisk == myframe.iloc[i]["conditionalRisk1"]:
            minRiskList.append(1)
        elif  minRisk == myframe.iloc[i]["conditionalRisk2"]:
            minRiskList.append(2)
        elif  minRisk == myframe.iloc[i]["conditionalRisk3"]:
            minRiskList.append(3)   
'''
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda0.75"],myframe.iloc[i]["ConditionalRisk2Lamda0.75"],myframe.iloc[i]["ConditionalRisk3Lamda0.75"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)





myframe["MinRiskLamda0.75"] = minRiskList


myRiskFrame2 = myframe[["row","ConditionalRisk1Lamda0.75","ConditionalRisk2Lamda0.75","ConditionalRisk3Lamda0.75","MinRiskLamda0.75"]]
display(myRiskFrame2)




overallRiskLamda75= myframe["MinRiskLamda0.75"].sum()

print("The overall risk for Lamda is 0.75 is below:",overallRiskLamda75)






In [ ]:
# STEP 5 lambda is 0.5 for myPosteriorFrame2
lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 0.5
lossClass1GivenClass3 = 0.5
lossClass2GivenClass1 = 0.5
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 0.5
lossClass3GivenClass1 = 0.5
lossClass3GivenClass2 = 0.5
lossClass3GivenClass3 = 0

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)
    #print(currentConditionalRisk)

myframe["ConditionalRisk1Lamda0.5"] = conditionalRisk1
myframe["ConditionalRisk2Lamda0.5"] = conditionalRisk2
myframe["ConditionalRisk3Lamda0.5"] = conditionalRisk3

minRiskList = list()
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda0.5"],myframe.iloc[i]["ConditionalRisk2Lamda0.5"],myframe.iloc[i]["ConditionalRisk3Lamda0.5"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)       



myframe["MinRiskLamda0.5"] = minRiskList


myRiskFrame3 = myframe[["row","ConditionalRisk1Lamda0.5","ConditionalRisk2Lamda0.5","ConditionalRisk3Lamda0.5","MinRiskLamda0.5"]]
display(myRiskFrame3)


overallRiskLamda5= myframe["MinRiskLamda0.5"].sum()

print("The overall risk for Lamda is 0.5 is below:",overallRiskLamda5)

In [ ]:
# STEP 5 lambda is 0.25 for myPosteriorFrame2
lossClass1GivenClass1 = 0
lossClass1GivenClass2 = 0.25
lossClass1GivenClass3 = 0.25
lossClass2GivenClass1 = 0.25
lossClass2GivenClass2 = 0
lossClass2GivenClass3 = 0.25
lossClass3GivenClass1 = 0.25
lossClass3GivenClass2 = 0.25
lossClass3GivenClass3 = 0

rows = myframe.shape[0]
print(rows)

conditionalRisk1 = []
conditionalRisk2 = []
conditionalRisk3 = []

currentConditionalRisk = None

for i in range(rows):
    action = myframe.iloc[i]["Class"]

    currentConditionalRisk = lossClass1GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass1GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass1GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk1.append(currentConditionalRisk)
    currentConditionalRisk = lossClass2GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass2GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass2GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk2.append(currentConditionalRisk)
    currentConditionalRisk = lossClass3GivenClass1*myframe.iloc[i]["PosteriorProbability1"] + lossClass3GivenClass2*myframe.iloc[i]["PosteriorProbability2"] + lossClass3GivenClass3*myframe.iloc[i]["PosteriorProbability3"]
    conditionalRisk3.append(currentConditionalRisk)
    #print(currentConditionalRisk)

myframe["ConditionalRisk1Lamda0.25"] = conditionalRisk1
myframe["ConditionalRisk2Lamda0.25"] = conditionalRisk2
myframe["ConditionalRisk3Lamda0.25"] = conditionalRisk3

minRiskList = list()
minRiskList = list()
for i in range(rows):
        riskList = [myframe.iloc[i]["ConditionalRisk1Lamda0.25"],myframe.iloc[i]["ConditionalRisk2Lamda0.25"],myframe.iloc[i]["ConditionalRisk3Lamda0.25"]]
        minRisk = min(riskList)
        minRiskList.append(minRisk)         



myframe["MinRiskLamda0.25"] = minRiskList


myRiskFrame4= myframe[["row","ConditionalRisk1Lamda0.25","ConditionalRisk2Lamda0.25","ConditionalRisk3Lamda0.25","MinRiskLamda0.25"]]
display(myRiskFrame4)


overallRiskLamda25= myframe["MinRiskLamda0.25"].sum()

print("The overall risk for Lamda is 0.25 is below:",overallRiskLamda25)

In [ ]:
# Printing the frame with lamda for minimum bayes risk for myPosteriorFrame1




minimum= overallRiskLamda1
minLamda = 1



if(overallRiskLamda75 < minimum):
    minimum = overallRiskLamda75
    minLamda = 0.75
if(overallRiskLamda5 < minimum):
    minimum = overallRiskLamda5
    minLamda = 0.5
if(overallRiskLamda25 < minimum):
    minLamda = 0.25
    minimum = overallRiskLamda25    

    
print("The Lamda for minimum Bayes Risk is")
display(minLamda)

End of Step 5

### STEP 6 - Calculate the discriminant Function (20 Points)

6.1 Case I: $\mathbb{E}_{i} = \sigma^2 * I$

Consider the case where features are statistically independent, and each feature has the same variance, i.e $ \sigma^2 $. (10 Points)



6.2 Case II: $\mathbb{E}_{i} = \mathbb{E} $

 Consider the case where the covariance matrices for all of the classes are identical but otherwise arbitrary.(10 Points)

Note: we would like you see creating a function with class specific mean and variance as the parameters and then compute the function for your chosen example dataset. Sample mean can be used for your class specific mean and prior can be computed from the collection, we have discussed the code in class. You can choose a range of values for $ \sigma^2 $ and $\mathbb{E}$ to generate the discriminant functions, so possible choices could be values around the sample variance or sample coveriance matrices. 

In [ ]:
# Helper Function to calculate prior and mean vec

def getPrior(reqClass, class1, class2,class3):
    #for binary classifiers
    return float(len(reqClass[0]))/float(len(class1[0]) + len(class2[0]) + len(class3[0]))
       
def getmeanVec(train):
    n = len(train)
    meanvec = np.zeros(n)
    for i in range(len(train)):
        meanvec[i] = np.mean(train[i])
    return meanvec

In [ ]:
# STEP 6
def discriminant_function(x_vec, cov_mat, mu_vec):
    """
    Calculates the value of the discriminant function for a dx1 dimensional
    sample given the covariance matrix and mean vector.
    
    Keyword arguments:
        x_vec: A dx1 dimensional numpy array representing the sample.
        cov_mat: numpy array of the covariance matrix.
        mu_vec: dx1 dimensional numpy array of the sample mean.
    
    Returns a float value as result of the discriminant function.
    
    """
    
    W_i = (-1/2) * np.linalg.inv(cov_mat)
    assert(W_i.shape[0] > 1 and W_i.shape[1] > 1), 'W_i must be a matrix'
    
    w_i = np.linalg.inv(cov_mat).dot(mu_vec)
    assert(w_i.shape[0] > 1 and w_i.shape[1] == 1), 'w_i must be a column vector'
    
    omega_i_p1 = (((-1/2) * (mu_vec).T).dot(np.linalg.inv(cov_mat))).dot(mu_vec)
    omega_i_p2 = (-1/2) * np.log(np.linalg.det(cov_mat))
    omega_i = omega_i_p1 - omega_i_p2
    assert(omega_i.shape == (1, 1)), 'omega_i must be a scalar'
    
    g = ((x_vec.T).dot(W_i)).dot(x_vec) + (w_i.T).dot(x_vec) + omega_i
    return float(g)
def discriminant_function1(x_vec,variance,mu_vec,prior):
    """
    Calculates the value of the discriminant function for a dx1 dimensional
    sample given the covariance matrix and mean vector.
    
    Keyword arguments:
        x_vec: A dx1 dimensional numpy array representing the sample.
        cov_mat: numpy array of the covariance matrix.
        mu_vec: dx1 dimensional numpy array of the sample mean.

    Returns a float value as result of the discriminant function.
    
    """
    x_vec = x_vec.reshape(-1,1)
    mu_vec = mu_vec.reshape(-1,1)

    W_i = (1/np.square(variance))*(((mu_vec).T))
    #W_i = (1/variance*variance)
    omega_i_p1 = (((-1/(2*np.square(variance))) * (mu_vec).T).dot(mu_vec)) + np.log(prior)
     
    
    g = (W_i).dot(x_vec) + omega_i_p1
    return float(g)

def discriminant_function2(x_vec, cov_mat, mu_vec,prior):
    """
    Calculates the value of the discriminant function for a dx1 dimensional
    sample given the covariance matrix and mean vector.
    
    Keyword arguments:
        x_vec: A dx1 dimensional numpy array representing the sample.
        cov_mat: numpy array of the covariance matrix.
        mu_vec: dx1 dimensional numpy array of the sample mean.

    Returns a float value as result of the discriminant function.
    
    """
    x_vec = x_vec.reshape(-1,1)
    mu_vec = mu_vec.reshape(-1,1)
    
    W_i = np.linalg.inv(cov_mat).dot(mu_vec)
    
    assert(W_i.shape[0] > 1 and W_i.shape[1] == 1), 'w_i must be a column vector'
    
    omega_i_p1 = (((-1/2) * (mu_vec).T).dot(np.linalg.inv(cov_mat))).dot(mu_vec)+ np.log(prior)
    
    
    g = (W_i.T).dot(x_vec)  + omega_i_p1
    return float(g)    


In [ ]:
# Creating data to use for discriminant function 1 from  output of training set 1 therefore we will use myPosteriorFrame1
train1a = x_subset_1.loc[x_subset_1['species'] == 'Iris-versicolor']
train2a = x_subset_1.loc[x_subset_1['species'] == 'Iris-virginica']
train3a = x_subset_1.loc[x_subset_1['species'] == 'Iris-setosa']

train1a = train1a.drop('species',axis =1)
train2a = train2a.drop('species',axis =1)
train3a = train3a.drop('species',axis =1)


test1a = x_full_test.drop('species',axis =1)

train1a = np.asarray(train1a.values.tolist()).transpose()
train2a = np.asarray(train2a.values.tolist()).transpose()
train3a = np.asarray(train3a.values.tolist()).transpose()



In [ ]:
# Discriminant Function Σ𝑖 = 𝜎2𝑰  for training set 1
# Here we will calculate discriminant function using this case 1 for every class . So in total we will have 3 discriminant function for 3 classes
# The discriminant function for class 1 will be given by discriminantGiven1. Simmillarly discriminantGiven2 for class 2 and simmillarly for class 3.


myframe1 = myPosteriorFrame1.copy()

from statistics import variance


cov1 = np.cov(train1a)
cov2 = np.cov(train2a)
cov3 = np.cov(train3a)

variance1 = cov1[0][0]
variance2 = cov2[0][0] 
variance3 = cov3[0][0]  

meanvec1 = getmeanVec(train1a)
meanvec2 = getmeanVec(train2a)
meanvec3 = getmeanVec(train3a)

prior1 = getPrior(train1a,train1a, train2a, train3a)
prior2 = getPrior(train2a,train1a, train2a, train3a)
prior3 = getPrior(train3a,train1a, train2a, train3a)



discriminantGiven1 = []
discriminantGiven2 = []
discriminantGiven3 = []


rows = myframe1.shape[0]




for i in range(rows):
    discriminant1 = discriminant_function1(myframe1.iloc[i]["row"],variance1, meanvec1,prior1)
    discriminantGiven1.append(discriminant1)
    discriminant2 = discriminant_function1(myframe1.iloc[i]["row"],variance2, meanvec2,prior2)
    discriminantGiven2.append(discriminant2)
    discriminant3 = discriminant_function1(myframe1.iloc[i]["row"],variance3, meanvec3,prior3)
    discriminantGiven3.append(discriminant3)


myframe1["discriminantGiven1"] = discriminantGiven1
myframe1["discriminantGiven2"] = discriminantGiven2
myframe1["discriminantGiven3"] = discriminantGiven3        

display(myframe1)


   

















In [ ]:
# Discriminant Function Σ𝑖 = Σ, for training set 1

discriminantGiven1 = []
discriminantGiven2 = []
discriminantGiven3 = []

myframe2 = myframe1

rows = myframe2.shape[0]




for i in range(rows):
    discriminant1 = discriminant_function2(myframe.iloc[i]["row"],cov1, meanvec1,prior1)
    discriminantGiven1.append(discriminant1)
    discriminant2 = discriminant_function2(myframe.iloc[i]["row"],cov2, meanvec2,prior2)
    discriminantGiven2.append(discriminant2)
    discriminant3 = discriminant_function2(myframe.iloc[i]["row"],cov3, meanvec3,prior3)
    discriminantGiven3.append(discriminant3)

myframe2["discriminantGiven1"] = discriminantGiven1
myframe2["discriminantGiven2"] = discriminantGiven2
myframe2["discriminantGiven3"] = discriminantGiven3        

display(myframe2)
    


In [ ]:
# Creating data to use for discriminant function 1 from output of training set 2 therefore we will use myPosteriorFrame2
train1a = x_subset_2.loc[x_subset_2['species'] == 'Iris-versicolor']
train2a = x_subset_2.loc[x_subset_2['species'] == 'Iris-virginica']
train3a = x_subset_2.loc[x_subset_2['species'] == 'Iris-setosa']

train1a = train1a.drop('species',axis =1)
train2a = train2a.drop('species',axis =1)
train3a = train3a.drop('species',axis =1)


test1a = x_full_test.drop('species',axis =1)

train1a = np.asarray(train1a.values.tolist()).transpose()
train2a = np.asarray(train2a.values.tolist()).transpose()
train3a = np.asarray(train3a.values.tolist()).transpose()

In [ ]:
# Discriminant Function Σ𝑖 = 𝜎2𝑰  for training set 2 therefore we will use myPosteriorFrame2


myframe1 = myPosteriorFrame2.copy()

from statistics import variance


cov1 = np.cov(train1a)
cov2 = np.cov(train2a)
cov3 = np.cov(train3a)

variance1 = cov1[0][0]
variance2 = cov2[0][0] 
variance3 = cov3[0][0]  

meanvec1 = getmeanVec(train1a)
meanvec2 = getmeanVec(train2a)
meanvec3 = getmeanVec(train3a)

prior1 = getPrior(train1a,train1a, train2a, train3a)
prior2 = getPrior(train2a,train1a, train2a, train3a)
prior3 = getPrior(train3a,train1a, train2a, train3a)



discriminantGiven1 = []
discriminantGiven2 = []
discriminantGiven3 = []


rows = myframe1.shape[0]




for i in range(rows):
    discriminant1 = discriminant_function1(myframe1.iloc[i]["row"],variance1, meanvec1,prior1)
    discriminantGiven1.append(discriminant1)
    discriminant2 = discriminant_function1(myframe1.iloc[i]["row"],variance2, meanvec2,prior2)
    discriminantGiven2.append(discriminant2)
    discriminant3 = discriminant_function1(myframe1.iloc[i]["row"],variance3, meanvec3,prior3)
    discriminantGiven3.append(discriminant3)


myframe1["discriminantGiven1"] = discriminantGiven1
myframe1["discriminantGiven2"] = discriminantGiven2
myframe1["discriminantGiven3"] = discriminantGiven3        

display(myframe1)


   

















In [ ]:
# Discriminant Function Σ𝑖 = Σ, for training set 2 
discriminantGiven1 = []
discriminantGiven2 = []
discriminantGiven3 = []

myframe2 = myframe1

rows = myframe2.shape[0]




for i in range(rows):
    discriminant1 = discriminant_function2(myframe.iloc[i]["row"],cov1, meanvec1,prior1)
    discriminantGiven1.append(discriminant1)
    discriminant2 = discriminant_function2(myframe.iloc[i]["row"],cov2, meanvec2,prior2)
    discriminantGiven2.append(discriminant2)
    discriminant3 = discriminant_function2(myframe.iloc[i]["row"],cov3, meanvec3,prior3)
    discriminantGiven3.append(discriminant3)

myframe2["discriminantGiven1"] = discriminantGiven1
myframe2["discriminantGiven2"] = discriminantGiven2
myframe2["discriminantGiven3"] = discriminantGiven3        

display(myframe2)
    

#### STEP 7 - Evaluate the Model ( 15 Points)
7.1 Plot the AUC - ROC Curve. (5 Points)

7.2 Evaluate the Precision, Recall, F1-Score. (10 Points)
N


In [ ]:
# For My Posterior Frame 1
# Will be using Try Catch if there is 0 in numerator or denominator
# class1Given1 is that predicted is 1 and actual is also 1 , class1Given2 is predicted is 1 but actual is two
# Here I am considering Iris VersiColor as 1, Iris Virginica as 2,Iris Setosa as 3
# I am plotting the auc curve for my first class which is Iris VersiColor

import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
plt.style.use('ggplot')

test1 = myPosteriorFrame1.copy()

y_score = []
y_true = y_full_test
y_true1 = np.where(y_true == 'Iris-versicolor',1,0)
y_score1 = []

class1Given1 = 0
class1Given2 = 0
class1Given3= 0
class2Given1 = 0
class2Given2 = 0
class2Given3= 0
class3Given1 = 0
class3Given2 = 0
class3Given3 = 0

data = list()

count = test1.shape[0]

for row in range(count):
       classPredicted = test1.iloc[row]["Class"]
       if classPredicted == 1 and y_true.iloc[row] == "Iris-versicolor":
              class1Given1 +=1
              y_score1.append(1)
       else:
              y_score1.append(0)       
              if classPredicted == 1 and y_true.iloc[row] == "Iris-virginica":
                     class1Given2 +=1        
              elif classPredicted == 1 and y_true.iloc[row] == "Iris-setosa":
                     class1Given3 +=1
              elif classPredicted == 2 and y_true.iloc[row] == "Iris-versicolor":
                     class2Given1 +=1
              elif classPredicted == 2 and y_true.iloc[row] == "Iris-virginica":
                     class2Given2 +=1        
              elif classPredicted == 2 and y_true.iloc[row] == "Iris-setosa":
                     class2Given3 +=1
              elif classPredicted == 3 and y_true.iloc[row] == "Iris-versicolor":
                     class3Given1 +=1
              elif classPredicted == 3 and y_true.iloc[row] == "Iris-virginica":
                     class3Given2 +=1        
              elif classPredicted == 3 and y_true.iloc[row] == "Iris-setosa":
                     class3Given3 +=1



try:
       Precision1 = class1Given1/(class1Given1+class1Given2+class1Given3)
except:
       Precision1 = 0     
try:
       Precision2 = class2Given2/(class2Given1+class2Given2+class2Given3)
except:
       Precision2 = 0 
try:
      Precision3 = class3Given3/(class3Given1+class3Given3+class3Given2)
except: 
       Precision3 = 0      


try:
       Recall1 = class1Given1/(class2Given1+class3Given1+class1Given1)
except:
       Recall1 = 0      
try:
       Recall2 = class2Given2/(class1Given2+class3Given2+class2Given2)
except:
       Recall2 = 0
try:
      Recall3 = class3Given3/(class1Given3+class2Given3+class3Given3)
except: 
       Recall3 = 0



print("Below is the ROC AUC Curve for my Posterior Frame 1")
fpr , tpr , _ = metrics.roc_curve(y_true1, y_score1)

plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
print(f'AUC score: {metrics.roc_auc_score(y_true1,y_score1)}')





print("Precision for Iris Versi color that is our class 1 is",Precision1)
print("Precision for Iris Verginica color that is our class 2 is",Precision2)
print("Precision for Iris Iris Setosa color that is our class 3 is",Precision3)

print("Recall for Iris Versi color that is our class 1 is",Recall1)
print("Recall for Iris Verginica color that is our class 2 is",Recall2)
print("Recall for Iris Iris Setosa color that is our class 3 is",Recall3)


# The formula for f1 score that I am using is (2*((Precision*Recall)/(Precision+Recall)))

print("The F1 Score for Iris Versi color which is our clas 1 is",(2*((Precision1*Recall1)/(Precision1+Recall1))))
print("The F1 Score for Iris Verginica color which is our clas 2 is",(2*((Precision2*Recall2)/(Precision2+Recall2))))
print("The F1 Score for Iris Setosa color which is our clas 3 is",(2*((Precision3*Recall3)/(Precision3+Recall3))))


print("Below is the confusion Matrix")

print(class1Given1,class2Given1,class3Given1)
print(class1Given2,class2Given2,class3Given2)
print(class1Given3,class2Given3,class3Given3)


    


In [ ]:
# For My Posterior Frame 2
# Will be using Try Catch if there is 0 in numerator or denominator

import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from itertools import cycle
plt.style.use('ggplot')

test2 = myPosteriorFrame2.copy()

y_score = []
y_true = y_full_test
y_true1 = np.where(y_true == 'Iris-versicolor',1,0)
y_score2 = []

class1Given1 = 0
class1Given2 = 0
class1Given3= 0
class2Given1 = 0
class2Given2 = 0
class2Given3= 0
class3Given1 = 0
class3Given2 = 0
class3Given3 = 0

data = list()
count = test2.shape[0]



for row in range(count):
       classPredicted = test2.iloc[row]["Class"]
       if classPredicted == 1 and y_true.iloc[row] == "Iris-versicolor":
              class1Given1 +=1
              y_score2.append(1)
       else:
              y_score2.append(0)       
              if classPredicted == 1 and y_true.iloc[row] == "Iris-virginica":
                     class1Given2 +=1        
              elif classPredicted == 1 and y_true.iloc[row] == "Iris-setosa":
                     class1Given3 +=1
              elif classPredicted == 2 and y_true.iloc[row] == "Iris-versicolor":
                     class2Given1 +=1
              elif classPredicted == 2 and y_true.iloc[row] == "Iris-virginica":
                     class2Given2 +=1        
              elif classPredicted == 2 and y_true.iloc[row] == "Iris-setosa":
                     class2Given3 +=1
              elif classPredicted == 3 and y_true.iloc[row] == "Iris-versicolor":
                     class3Given1 +=1
              elif classPredicted == 3 and y_true.iloc[row] == "Iris-virginica":
                     class3Given2 +=1        
              elif classPredicted == 3 and y_true.iloc[row] == "Iris-setosa":
                     class3Given3 +=1







try:
       Precision1 = class1Given1/(class1Given1+class1Given2+class1Given3)
except:
       Precision1 = 0     
try:
       Precision2 = class2Given2/(class2Given1+class2Given2+class2Given3)
except:
       Precision2 = 0 
try:
      Precision3 = class3Given3/(class3Given1+class3Given3+class3Given2)
except: 
       Precision3 = 0      


try:
       Recall1 = class1Given1/(class2Given1+class3Given1+class1Given1)
except:
       Recall1 = 0      
try:
       Recall2 = class2Given2/(class1Given2+class3Given2+class2Given2)
except:
       Recall2 = 0
try:
      Recall3 = class3Given3/(class1Given3+class2Given3+class3Given3)
except: 
       Recall3 = 0




print("Below is the ROC AUC Curve for my Posterior Frame 2")
fpr , tpr , _ = metrics.roc_curve(y_true1, y_score2)

plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
print(f'AUC score: {metrics.roc_auc_score(y_true1,y_score2)}')





print("Precision for Iris Versi color that is our class 1 is",Precision1)
print("Precision for Iris Verginica color that is our class 2 is",Precision2)
print("Precision for Iris Iris Setosa color that is our class 3 is",Precision3)

print("Recall for Iris Versi color that is our class 1 is",Recall1)
print("Recall for Iris Verginica color that is our class 2 is",Recall2)
print("Recall for Iris Iris Setosa color that is our class 3 is",Recall3)

print("The F1 Score for Iris Versi color which is our clas 1 is",(2*((Precision1*Recall1)/(Precision1+Recall1))))
print("The F1 Score for Iris Verginica color which is our clas 2 is",(2*((Precision2*Recall2)/(Precision2+Recall2))))
print("The F1 Score for Iris Setosa color which is our clas 3 is",(2*((Precision3*Recall3)/(Precision3+Recall3))))


print("Below is the confusion Matrix")

print(class1Given1,class2Given1,class3Given1)
print(class1Given2,class2Given2,class3Given2)
print(class1Given3,class2Given3,class3Given3)




    


## Useful Library references

  [test_train_split function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split)


[Matplotlib Documentation](https://matplotlib.org/)

## Slides Reference 

  [Bayes Decision Theory](https://ublearns.buffalo.edu/ultra/courses/_192160_1/cl/outline)
